In [1]:
import pandas as pd
from tqdm import tqdm
import pickle as pkl

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install datasets loralib sentencepiece

In [ ]:
!pip install bitsandbytes accelerate xformers einops

In [4]:
from torch.fx import symbolic_trace

In [ ]:
from transformers import AutoTokenizer, AutoModel, LlamaModel, LlamaTokenizer,LlamaForCausalLM
import transformers
import torch
model = "meta-llama/Llama-2-7b-chat-hf"
hf_auth = '********************************' # Huggingface Authorization token is required which can be accessed upon request from this link https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True, use_auth_token=hf_auth)
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=hf_auth)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    token=hf_auth,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer=tokenizer
)

In [7]:
#input your data in csv format
test_data=pd.read_csv("/notebooks/data/fever/3_class/1_shot.csv")

In [ ]:
zip_new_test=dict(zip(list(test_data['cl_id']),list(test_data['claim'])))
zip_test_label=dict(zip(list(test_data['cl_id']),list(test_data['label'])))

In [46]:
# from transformers import pipeline, set_seed
set_seed(42)
zip_new_test_temp={}
for i,j in enumerate(tqdm(zip_new_test)):
    claim=zip_new_test[j]
    part1='Evaluate the validity of the claim'
    part2="'{}'".format(claim)
    part3='by determining if it is true or false or lack of information for confirmation. Answer:'
    wh_sen=part1+' '+part2+' '+part3
    zip_new_test_temp[j]=wh_sen

100%|██████████| 19755/19755 [00:00<00:00, 732826.34it/s]


In [ ]:
set_seed(42)
sequences = pipeline(
    '''Evaluate the validity of the claim 'A polynomial is a convoluted code.' by determining if it is true or false or not enough information (NEI). Answer:''',
    # '''Fact verification involves evaluating statements based on available evidence to determine their truth, falsity, or insufficient supporting information.Now please evaluate the validity of the claim by determining if it is true or false or does it lack of sufficient information for confirmation. "Input": Sora (Kingdom Hearts) and Donald Duck went on an adventure.  ''',
    # '''Generate a valid label from the collection of these given labels i.e. ['True', 'False', 'NEI'] for the given claim 'Michelin Guides are a series of restaurant guide books.'. Answer:''',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=100, temperature=0.1
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [45]:
sequences

[{'generated_text': "Evaluate the validity of the claim 'Lizzy Caplan reviewed the television show True Blood.\n10' by determining if it is true or false or lack of information for confirmation. Answer: True"}]